In [1]:
# notebooks/items.ipynb

# RPGシミュレーター - アイテムシステム
# 
# このノートブックでは、RPGシミュレーターのアイテムシステムを実装します。
# 回復アイテムや装備品の基本クラスを定義し、キャラクターのインベントリシステムを実装します。

import json
import os
import random
from typing import Dict, List, Optional, Any, Union

# キャラクタークラスを読み込む
%run notebooks/characters.ipynb

# ## アイテムタイプの定義
ITEM_TYPE_CONSUMABLE = "consumable"  # 消費アイテム（使い切り）
ITEM_TYPE_EQUIPMENT = "equipment"    # 装備品
ITEM_TYPE_KEY = "key"                # キーアイテム（後々の拡張用）

# 装備品のスロット定義
EQUIP_SLOT_WEAPON = "weapon"         # 武器
EQUIP_SLOT_ARMOR = "armor"           # 防具
EQUIP_SLOT_ACCESSORY = "accessory"   # アクセサリー

# ## 基本アイテムクラス
class Item:
    """RPGシミュレーターの基本アイテムクラス"""
    
    def __init__(self, id: str, name: str, description: str, item_type: str, price: int = 0):
        """アイテムの初期化
        
        Args:
            id: アイテムの一意識別子
            name: アイテム名
            description: アイテムの説明
            item_type: アイテムタイプ
            price: アイテムの価格（販売/購入用）
        """
        self.id = id
        self.name = name
        self.description = description
        self.item_type = item_type
        self.price = price
    
    def use(self, user: Character, target: Optional[Character] = None) -> Dict[str, Any]:
        """アイテムを使用する基本メソッド（オーバーライド用）
        
        Args:
            user: アイテムを使用するキャラクター
            target: アイテムの使用対象（省略可能）
            
        Returns:
            使用結果の情報を含む辞書
        """
        return {
            "success": False,
            "message": f"{self.name}は使用できません。"
        }
    
    def to_dict(self) -> Dict[str, Any]:
        """アイテム情報を辞書形式で取得する
        
        Returns:
            アイテム情報の辞書
        """
        return {
            "id": self.id,
            "name": self.name,
            "description": self.description,
            "item_type": self.item_type,
            "price": self.price
        }

# ## 消費アイテムクラス
class ConsumableItem(Item):
    """使用して効果を発揮する消費アイテムクラス"""
    
    def __init__(self, id: str, name: str, description: str, price: int = 0, 
                 effect_type: str = "heal_hp", effect_value: int = 0):
        """消費アイテムの初期化
        
        Args:
            id: アイテムの一意識別子
            name: アイテム名
            description: アイテムの説明
            price: アイテムの価格
            effect_type: 効果タイプ（heal_hp, heal_mp, etc.）
            effect_value: 効果量
        """
        super().__init__(id, name, description, ITEM_TYPE_CONSUMABLE, price)
        self.effect_type = effect_type
        self.effect_value = effect_value
    
    def use(self, user: Character, target: Optional[Character] = None) -> Dict[str, Any]:
        """消費アイテムを使用する
        
        Args:
            user: アイテムを使用するキャラクター
            target: アイテムの使用対象（未指定の場合は使用者自身）
            
        Returns:
            使用結果の情報を含む辞書
        """
        # ターゲットが指定されていない場合は使用者自身
        if target is None:
            target = user
        
        # キャラクターが戦闘不能の場合は使用できない
        if not target.is_alive:
            return {
                "success": False,
                "message": f"{target.name}は戦闘不能のため、{self.name}を使用できません。"
            }
        
        result = {
            "success": True,
            "user": user.name,
            "target": target.name,
            "item_name": self.name
        }
        
        # 効果タイプに応じた処理
        if self.effect_type == "heal_hp":
            # HP回復
            heal_amount = target.heal(self.effect_value)
            result["effect_type"] = "heal_hp"
            result["amount"] = heal_amount
            result["message"] = f"{target.name}のHPが{heal_amount}回復した！"
        
        elif self.effect_type == "heal_mp":
            # MP回復
            mp_amount = target.recover_mp(self.effect_value)
            result["effect_type"] = "heal_mp"
            result["amount"] = mp_amount
            result["message"] = f"{target.name}のMPが{mp_amount}回復した！"
            
        else:
            # その他の効果（将来的な拡張用）
            result["effect_type"] = self.effect_type
            result["amount"] = self.effect_value
            result["message"] = f"{self.name}を使用したが、よくわからない効果だった。"
        
        return result
    
    def to_dict(self) -> Dict[str, Any]:
        """アイテム情報を辞書形式で取得する"""
        base_dict = super().to_dict()
        base_dict.update({
            "effect_type": self.effect_type,
            "effect_value": self.effect_value
        })
        return base_dict

# ## 装備品クラス
class EquipmentItem(Item):
    """装備して能力を上昇させる装備品クラス"""
    
    def __init__(self, id: str, name: str, description: str, price: int = 0,
                 slot: str = EQUIP_SLOT_WEAPON, stat_bonuses: Dict[str, int] = None):
        """装備品の初期化
        
        Args:
            id: アイテムの一意識別子
            name: アイテム名
            description: アイテムの説明
            price: アイテムの価格
            slot: 装備スロット
            stat_bonuses: 能力上昇値の辞書（strength, defense, etc.）
        """
        super().__init__(id, name, description, ITEM_TYPE_EQUIPMENT, price)
        self.slot = slot
        self.stat_bonuses = stat_bonuses or {}
    
    def use(self, user: Character, target: Optional[Character] = None) -> Dict[str, Any]:
        """装備品を装備する
        
        Args:
            user: 装備するキャラクター
            target: 未使用引数（API一貫性のため）
            
        Returns:
            装備結果の情報を含む辞書
        """
        # ここではシンプルに装備可能という結果だけを返す
        # 実際の装備処理はPlayerCharacterクラスで行う
        return {
            "success": True,
            "message": f"{user.name}は{self.name}を装備できます。",
            "equip_slot": self.slot
        }
    
    def get_stat_bonus(self, stat_name: str) -> int:
        """特定のステータスに対するボーナス値を取得する
        
        Args:
            stat_name: ステータス名
            
        Returns:
            ボーナス値（なければ0）
        """
        return self.stat_bonuses.get(stat_name, 0)
    
    def to_dict(self) -> Dict[str, Any]:
        """アイテム情報を辞書形式で取得する"""
        base_dict = super().to_dict()
        base_dict.update({
            "slot": self.slot,
            "stat_bonuses": self.stat_bonuses
        })
        return base_dict

# ## アイテム生成関数
def create_sample_items() -> Dict[str, Item]:
    """サンプルアイテムを作成する
    
    Returns:
        アイテムIDをキー、Itemオブジェクトを値とする辞書
    """
    items = {}
    
    # 回復アイテム
    items["potion"] = ConsumableItem(
        id="potion",
        name="ポーション",
        description="HPを50回復する薬",
        price=50,
        effect_type="heal_hp",
        effect_value=50
    )
    
    items["hi_potion"] = ConsumableItem(
        id="hi_potion",
        name="ハイポーション",
        description="HPを150回復する薬",
        price=300,
        effect_type="heal_hp",
        effect_value=150
    )
    
    items["ether"] = ConsumableItem(
        id="ether",
        name="エーテル",
        description="MPを30回復する薬",
        price=100,
        effect_type="heal_mp",
        effect_value=30
    )
    
    # 装備品
    items["bronze_sword"] = EquipmentItem(
        id="bronze_sword",
        name="ブロンズソード",
        description="銅で作られた剣。攻撃力+5",
        price=240,
        slot=EQUIP_SLOT_WEAPON,
        stat_bonuses={"strength": 5}
    )
    
    items["iron_armor"] = EquipmentItem(
        id="iron_armor",
        name="鉄の鎧",
        description="鉄で作られた鎧。防御力+8",
        price=400,
        slot=EQUIP_SLOT_ARMOR,
        stat_bonuses={"defense": 8}
    )
    
    items["silver_ring"] = EquipmentItem(
        id="silver_ring",
        name="銀の指輪",
        description="銀製の指輪。魔法攻撃力+3、素早さ+2",
        price=350,
        slot=EQUIP_SLOT_ACCESSORY,
        stat_bonuses={"magic": 3, "speed": 2}
    )
    
    return items

# ## PlayerCharacterクラスの拡張（インベントリと装備品）
# このコードはcharacters.ipynbに追加する必要があります
"""
PlayerCharacterクラスの拡張コードは別途追加します
"""

# ## テスト関数
def test_items():
    """アイテムシステムのテスト"""
    # サンプルキャラクターとアイテムを作成
    hero = create_sample_character()
    items = create_sample_items()
    
    print("==== アイテムシステムテスト ====")
    
    # インベントリの初期化
    hero.inventory = {}
    hero.equipment = {
        EQUIP_SLOT_WEAPON: None,
        EQUIP_SLOT_ARMOR: None,
        EQUIP_SLOT_ACCESSORY: None
    }
    
    # アイテムの追加
    hero.add_item("potion", 3)
    hero.add_item("ether", 2)
    hero.add_item("bronze_sword", 1)
    
    print(f"{hero.name}のインベントリ:")
    for item_id, count in hero.inventory.items():
        if item_id in items:
            print(f"- {items[item_id].name} x{count}")
    
    # アイテムの使用
    print("\n[ポーションのテスト]")
    hero.current_hp = 70  # HPを減らしておく
    result = hero.use_item("potion", items)
    print(result["message"])
    print(f"{hero.name}のHP: {hero.current_hp}/{hero.max_hp}")
    
    # 装備品の装備
    print("\n[装備品のテスト]")
    old_strength = hero.strength
    result = hero.use_item("bronze_sword", items)
    print(result["message"])
    print(f"攻撃力変化: {old_strength} -> {hero.strength}")
    
    # 装備情報の確認
    print("\n現在の装備:")
    equip_info = hero.get_equipment_info(items)
    for slot, item_name in equip_info.items():
        print(f"- {slot}: {item_name}")
    
    print("\n==== アイテムシステムテスト終了 ====")

if __name__ == "__main__":
    # テスト実行
    # test_items()
    pass

勇者のステータス:
name: 勇者
level: 1
hp: 100/100
mp: 50/50
strength: 15
defense: 10
magic: 8
speed: 12
is_alive: True
job: 戦士
exp: 0
next_level_exp: 100
exp_to_next: 100

敵のステータス:
name: スライム
level: 1
hp: 50/50
mp: 20/20
strength: 8
defense: 5
magic: 3
speed: 6
is_alive: True
enemy_type: 通常
exp_reward: 15

簡易戦闘テスト:
勇者がスライムに攻撃！
11のダメージを与えた！
スライムのHP: 39/50

スライムが勇者に攻撃！
3のダメージを与えた！
勇者のHP: 97/100
